In [8]:
import os
import numpy as np
import pandas as pd
import re
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from transformers import LayoutLMTokenizer, LayoutLMForTokenClassification
import pytesseract
from PIL import Image
import requests
from io import BytesIO
from albumentations import Compose, Resize, RandomCrop, RandomBrightnessContrast
import torch  # For CUDA
from sklearn.model_selection import train_test_split, StratifiedKFold
import mlflow  # For experiment tracking
import matplotlib.pyplot as plt
import sys
import os

# Assuming your script is in the same directory as src
sys.path.append(os.path.abspath("D:\Projects\Amazon-ML-Challenge-Team-AutoGen-\student_resource 3\src"))

from utils import download_images  # Assuming this function is defined in utils.py
from constants import entity_unit_map, allowed_units  # Importing the unit map

<>:24: SyntaxWarning: invalid escape sequence '\P'
<>:24: SyntaxWarning: invalid escape sequence '\P'
C:\Users\Sudee\AppData\Local\Temp\ipykernel_18324\771604961.py:24: SyntaxWarning: invalid escape sequence '\P'
  sys.path.append(os.path.abspath("D:\Projects\Amazon-ML-Challenge-Team-AutoGen-\student_resource 3\src"))
d:\python3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
TRAIN_CSV = 'dataset/train.csv'
TEST_CSV = 'dataset/test.csv'

In [10]:
def download_images(image_links, save_dir='images', batch_id=0):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    image_paths = []
    for i, link in enumerate(image_links):
        image_name = f'image_{batch_id}_{i}.jpg'
        image_path = os.path.join(save_dir, image_name)
        
        # Check if the image already exists (optional)
        if not os.path.exists(image_path):
            response = requests.get(link)
            image = Image.open(BytesIO(response.content))
            image.save(image_path)
        
        image_paths.append(image_path)
    
    return image_paths


In [11]:
from PIL import Image
import numpy as np
import albumentations as A

def preprocess_images(image_paths, target_size=(224, 224)):
    augment = A.Compose([
        A.Resize(height=target_size[0], width=target_size[1]),
        A.RandomBrightnessContrast(p=0.2)
    ])
    
    images = []
    for path in image_paths:
        # Open the image, convert to RGB, and convert to numpy array
        image = Image.open(path).convert('RGB')
        image = np.array(image)
        
        # Apply the augmentations
        augmented = augment(image=image)
        images.append(augmented['image'])
    
    return np.array(images)


In [12]:
def extract_text_from_images(image_paths):
    texts = []
    for path in image_paths:
        image = Image.open(path)
        text = pytesseract.image_to_string(image)
        texts.append(text)
    return texts

In [13]:
unit_abbreviations = {
    # For 'item_weight' and 'maximum_weight_recommendation'
    'gram': ['g', 'gr', 'gm', 'grams', 'grm'],
    'kilogram': ['kg', 'kilograms', 'kgs'],
    'milligram': ['mg', 'milligrams', 'mgs'],
    'microgram': ['µg', 'mcg', 'micrograms'],
    'ounce': ['oz', 'ounces', 'ozs'],
    'pound': ['lb', 'lbs', 'pounds'],
    'ton': ['t', 'tons', 'tonne', 'tonnes'],

    # For 'item_volume'
    'millilitre': ['ml', 'milliliters', 'millilitres'],
    'litre': ['l', 'lit', 'liters', 'litres'],
    'cubic_centimetre': ['cc', 'cm³', 'cubic centimeters', 'cubic centimetres'],
    'cubic_metre': ['m³', 'cubic meters', 'cubic metres'],
    'gallon': ['gal', 'gallons'],
    'quart': ['qt', 'quarts'],
    'pint': ['pt', 'pints'],
    'cup': ['c', 'cups'],

    # For 'voltage'
    'volt': ['v', 'volts'],
    'kilovolt': ['kv', 'kilovolts'],
    'millivolt': ['mv', 'millivolts'],

    # For 'wattage'
    'watt': ['w', 'watts'],
    'kilowatt': ['kw', 'kilowatts'],
    'megawatt': ['mw', 'megawatts'],
    'gigawatt': ['gw', 'gigawatts'],

    # For 'height', 'depth', and 'width'
    'millimetre': ['mm', 'millimeters', 'millimetres'],
    'centimetre': ['cm', 'centimeters', 'centimetres'],
    'metre': ['m', 'meters', 'metres'],
    'kilometre': ['km', 'kilometers', 'kilometres'],
    'inch': ['in', 'inches'],
    'foot': ['ft', 'feet'],
    'yard': ['yd', 'yards'],
    'mile': ['mi', 'miles'],

    # Other common units
    'degree_celsius': ['°C', 'C', 'degrees Celsius'],
    'degree_fahrenheit': ['°F', 'F', 'degrees Fahrenheit'],
    'calorie': ['cal', 'calories'],
    'kilocalorie': ['kcal', 'kcals'],
    'joule': ['j', 'joules'],
    'pascal': ['Pa', 'pascals'],
    'bar': ['bar', 'bars'],
    'psi': ['psi', 'pounds per square inch'],
    'newton': ['N', 'newtons'],
    'fluid_ounce': ['fl oz', 'fluid ounces'],
}

def standardize_unit(value, entity_name):
    for unit, abbreviations in unit_abbreviations.items():
        for abbr in abbreviations:
            if re.search(r'\b' + re.escape(abbr) + r'\b', value):
                standardized_value = re.sub(abbr, unit, value)
                return standardized_value
    return value  # If no abbreviation found, return the original value


In [14]:
def initialize_resnet_model():
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(1, activation='linear')(x)  # For regression
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # Mixed Precision
    policy = tf.keras.mixed_precision.Policy('mixed_float16')
    tf.keras.mixed_precision.set_global_policy(policy)
    
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    return model

In [15]:
def initialize_layoutlm_model(num_labels):
    tokenizer = LayoutLMTokenizer.from_pretrained('microsoft/layoutlm-base-uncased')
    model = LayoutLMForTokenClassification.from_pretrained('microsoft/layoutlm-base-uncased', num_labels=num_labels)
    return tokenizer, model

In [16]:
def predict_with_resnet(model, images):
    predictions = model.predict(images)
    return predictions.flatten()  # For regression, return raw predictions

def predict_with_layoutlm(model, tokenizer, texts):
    encodings = tokenizer(texts, truncation=True, padding=True, return_tensors='pt')
    outputs = model(**encodings)
    return np.argmax(outputs.logits.detach().numpy(), axis=1)


In [17]:
def generate_predictions(test_df, resnet_model, layoutlm_model, tokenizer):
    image_links = test_df['image_link'].tolist()
    image_paths = download_images(image_links)
    images = preprocess_images(image_paths)
    
    # Feature Extraction with ResNet
    resnet_predictions = predict_with_resnet(resnet_model, images)
    
    # Text Extraction and LayoutLM Predictions
    texts = extract_text_from_images(image_paths)
    cleaned_texts = [standardize_unit(text, None) for text in texts]  # Pass None for entity_name if not needed
    layoutlm_predictions = predict_with_layoutlm(layoutlm_model, tokenizer, cleaned_texts)
    
    # Combine predictions
    predictions = [f"{resnet_pred} {layoutlm_pred}" for resnet_pred, layoutlm_pred in zip(resnet_predictions, layoutlm_predictions)]
    return predictions

In [18]:
def save_predictions(test_df, predictions, submission_file='submission.csv'):
    test_df['prediction'] = predictions
    test_df.to_csv(submission_file, index=False)
    print(f"Predictions saved to {submission_file}")

In [19]:
def cumulative_training(train_df, model_save_path, batch_size=100, max_batches=30):
    best_mae = float('inf')  # Mean Absolute Error for regression
    for batch in range(1, max_batches + 1):
        print(f"Processing batch {batch}...")
        
        # Download and preprocess images for the current batch
        image_links = train_df['image_link'].head(batch * batch_size).tolist()
        image_paths = download_images(image_links, batch_id=batch)  # Pass batch_id
        images = preprocess_images(image_paths)
        
        # Extract and process labels
        labels = train_df['entity_value'].head(batch * batch_size).apply(lambda x: float(re.sub(r'[^\d.]', '', x))).values
        labels = labels[~np.isnan(labels)]  # Remove NaNs if any
        
        # Create TensorFlow dataset
        dataset = tf.data.Dataset.from_tensor_slices((images, labels))
        dataset = dataset.shuffle(buffer_size=1000).batch(batch_size)
        
        # Split dataset into training and validation sets
        dataset_size = len(images)
        validation_size = int(0.2 * dataset_size)
        train_size = dataset_size - validation_size
        
        train_dataset = dataset.take(train_size)
        validation_dataset = dataset.skip(train_size)
        
        # Initialize and compile the model for regression
        model = initialize_resnet_model()
        
        # Learning Rate Scheduling
        def lr_scheduler(epoch, lr):
            if epoch < 10:
                return lr
            else:
                return lr * tf.math.exp(-0.1)
        
        callbacks = [
            EarlyStopping(monitor='val_mae', patience=3, restore_best_weights=True),
            ModelCheckpoint(model_save_path, save_best_only=True, monitor='val_mae'),
            LearningRateScheduler(lr_scheduler)
        ]
        
        history = model.fit(
            train_dataset,
            epochs=5,
            validation_data=validation_dataset,  # Ensure validation data is provided
            callbacks=callbacks
        )
        
        # Evaluate model
        mae = min(history.history['val_mae'])
        print(f"Batch {batch} MAE: {mae:.4f}")
        
        if mae < best_mae:
            best_mae = mae
            best_model = load_model(model_save_path)
    
    print(f"Best MAE: {best_mae:.4f}")
    return best_model


: 

In [20]:
if __name__ == "__main__":
    # Adjust NUM_CLASSES as needed
    NUM_CLASSES = 30  # Example number of classes, adjust based on your task
    model_save_path = 'best_model.keras'
    
    # Load data
    train_df = pd.read_csv(TRAIN_CSV)
    test_df = pd.read_csv(TEST_CSV)
    
    # Cumulative Training
    best_model = cumulative_training(train_df, model_save_path, batch_size=100)
    
    # Initialize LayoutLM and ResNet models
    tokenizer, layoutlm_model = initialize_layoutlm_model(num_labels=NUM_CLASSES)
    resnet_model = load_model(model_save_path)
    
    # Generate predictions
    predictions = generate_predictions(test_df, resnet_model, layoutlm_model, tokenizer)
    
    # Save predictions
    save_predictions(test_df, predictions)

Processing batch 1...
Epoch 1/5
